In [47]:
%load_ext autoreload
%autoreload 2

import numpy as np
import os, sys
import sqlite3
import torch

sys.path.insert(0, '../src')
sys.path.insert(1, '..')

from main import get_model
from models.config import Config, IMAGE_TOKEN_IDS
import db_utils

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
sys.argv = ['notebooks/get_representations.ipynb',
            '--config', '../configs/models/qwen/Qwen2-VL-2B-Instruct-VisualFlow.yaml']

config = Config()

In [14]:
model = get_model(config.architecture, config)
# print(model.model)

Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00,  8.07it/s]


In [15]:
print(model.model)

Qwen2VLForConditionalGeneration(
  (visual): Qwen2VisionTransformerPretrainedModel(
    (patch_embed): PatchEmbed(
      (proj): Conv3d(3, 1280, kernel_size=(2, 14, 14), stride=(2, 14, 14), bias=False)
    )
    (rotary_pos_emb): VisionRotaryEmbedding()
    (blocks): ModuleList(
      (0-31): 32 x Qwen2VLVisionBlock(
        (norm1): LayerNorm((1280,), eps=1e-06, elementwise_affine=True)
        (norm2): LayerNorm((1280,), eps=1e-06, elementwise_affine=True)
        (attn): VisionSdpaAttention(
          (qkv): Linear(in_features=1280, out_features=3840, bias=True)
          (proj): Linear(in_features=1280, out_features=1280, bias=True)
        )
        (mlp): VisionMlp(
          (fc1): Linear(in_features=1280, out_features=5120, bias=True)
          (act): QuickGELUActivation()
          (fc2): Linear(in_features=5120, out_features=1280, bias=True)
        )
      )
    )
    (merger): PatchMerger(
      (ln_q): LayerNorm((1280,), eps=1e-06, elementwise_affine=True)
      (mlp): Seq

In [16]:
model.run(save_tokens=True)

Running forward hooks on data: 100%|██████████| 2/2 [00:00<00:00,  2.91it/s]


In [17]:
db_path = model.config.output_db
print(db_path)
connection = sqlite3.connect(db_path)
connection.row_factory = sqlite3.Row
cursor = connection.cursor()

../output/Qwen2-VL-2B-Instruct-VisualFlow.db


In [18]:
for r in cursor.execute("""
  SELECT id, image_id, name, image_path, prompt, label, layer, pooling_method, tensor_dim, tensor_shape,
         length(tensor) AS tensor_bytes
  FROM tensors
  LIMIT 20
"""):
    print(dict(r))

{'id': 1, 'image_id': None, 'name': 'Qwen/Qwen2-VL-2B-Instruct', 'image_path': '/n/fs/ac-aiexhibit/vlm-lens/data/test-images/black_in_blue.png', 'prompt': 'Describe the color in this image in one word.', 'label': None, 'layer': 'input_ids', 'pooling_method': None, 'tensor_dim': 1, 'tensor_shape': 'torch.Size([1, 129])', 'tensor_bytes': 2204}
{'id': 2, 'image_id': None, 'name': 'Qwen/Qwen2-VL-2B-Instruct', 'image_path': '/n/fs/ac-aiexhibit/vlm-lens/data/test-images/black_in_blue.png', 'prompt': 'Describe the color in this image in one word.', 'label': None, 'layer': 'visual.blocks.31', 'pooling_method': None, 'tensor_dim': 1280, 'tensor_shape': 'torch.Size([392, 1280])', 'tensor_bytes': 1004700}
{'id': 3, 'image_id': None, 'name': 'Qwen/Qwen2-VL-2B-Instruct', 'image_path': '/n/fs/ac-aiexhibit/vlm-lens/data/test-images/black_in_blue.png', 'prompt': 'Describe the color in this image in one word.', 'label': None, 'layer': 'visual.merger', 'pooling_method': None, 'tensor_dim': 1536, 'tensor

In [53]:
for r in cursor.execute("""
  SELECT id, name, label, image_path, layer, tensor_dim, tensor_shape,
         length(tensor) AS tensor_bytes
  FROM tensors
  WHERE layer = "visual.merger"
  LIMIT 20
"""):
    print(dict(r))

{'id': 3, 'name': 'Qwen/Qwen2-VL-2B-Instruct', 'label': None, 'image_path': '/n/fs/ac-aiexhibit/vlm-lens/data/test-images/black_in_blue.png', 'layer': 'visual.merger', 'tensor_dim': 1536, 'tensor_shape': 'torch.Size([98, 1536])', 'tensor_bytes': 302236}
{'id': 9, 'name': 'Qwen/Qwen2-VL-2B-Instruct', 'label': None, 'image_path': '/n/fs/ac-aiexhibit/vlm-lens/data/test-images/black_in_black.png', 'layer': 'visual.merger', 'tensor_dim': 1536, 'tensor_shape': 'torch.Size([98, 1536])', 'tensor_bytes': 302236}


In [ ]:
input_ids = db_utils.get_embeddings_by_layer(
    db_path=db_path,
    layer_name="input_ids",
    device="cuda"
)

post_llm_embeddings = db_utils.get_embeddings_by_layer(
    db_path=db_path,
    layer_name="model.norm",
    device="cuda"
)


input_ids = np.squeeze(np.array(tuple(map(list, zip(*input_ids)))[1])) # B x T
print(input_ids)
# print(torch.tensor(np.array(input_ids)).shape)

post_llm_embeddings = np.squeeze(np.array(tuple(map(list, zip(*post_llm_embeddings)))[1])) # B x T x D
# print(torch.tensor(np.array(post_llm_embeddings)).shape)



visual_embs, n_visual_embs = db_utils.extract_visual_embeddings(
    input_ids=input_ids,
    llm_embeddings=post_llm_embeddings,
    image_token_id=IMAGE_TOKEN_IDS['qwen'])

mean_embs = db_utils.compute_mean_embeddings(
    embeddings=visual_embs,
    n_embeddings=n_visual_embs)

db_utils.cosine_similarity_numpy(mean_embs, mean_embs)

# TODO: for each unique layer in DB (or is this saved in model.config?), get the embedding representations
# TODO: Create similarity matrix for each pair of layers (averaged over (B * (B-1) / 2)), and save a visual representation

[[151644   8948    198   2610    525    264  10950  17847     13 151645
     198 151644    872    198 151652 151655 151655 151655 151655 151655
  151655 151655 151655 151655 151655 151655 151655 151655 151655 151655
  151655 151655 151655 151655 151655 151655 151655 151655 151655 151655
  151655 151655 151655 151655 151655 151655 151655 151655 151655 151655
  151655 151655 151655 151655 151655 151655 151655 151655 151655 151655
  151655 151655 151655 151655 151655 151655 151655 151655 151655 151655
  151655 151655 151655 151655 151655 151655 151655 151655 151655 151655
  151655 151655 151655 151655 151655 151655 151655 151655 151655 151655
  151655 151655 151655 151655 151655 151655 151655 151655 151655 151655
  151655 151655 151655 151655 151655 151655 151655 151655 151655 151655
  151655 151655 151655 151653  74785    279   1894    304    419   2168
     304    825   3409     13 151645    198 151644  77091    198]
 [151644   8948    198   2610    525    264  10950  17847     13 15164

np.float64(0.9883939244138062)